### Dream Team:  Xingce Bao, Sohyeong Kim, Giulio Masinelli, Silvio Zanoli


# PART IV - Routing the travel

In this part, we are going to build a robust routing algorithm for routing that considers risk factors. 

We basically used the same idea from the Part 3 and here we put a constraint over the number of availble transfers during the travel. Then among the possible routes that it founds, we choose the 5 fastest routes. 

Then we visualize our suggested route in the google map using Google Map API. By doing this way we can easily see how we can reach a destination from Zurich HB.

In [1]:
import getpass
import os
import numpy as np
import pandas as pd
from datetime import timedelta
import pickle
import scipy.stats


First, we start by importing processed data created and stored from the previous sections. 

In [2]:
# Import the lists of stations within 10km from Zürich HB
station_data = pickle.load( open( "./data/train_station_id.p", "rb" ) )

# Import the processed actual data containing mean and variance
data_mean_variance_list = []
data_mean_variance_list.append(pd.read_csv('./data/monday_processed.csv'))
data_mean_variance_list.append(pd.read_csv('./data/tuesday_processed.csv'))
data_mean_variance_list.append(pd.read_csv('./data/wednesday_processed.csv'))
data_mean_variance_list.append(pd.read_csv('./data/thursday_processed.csv'))
data_mean_variance_list.append(pd.read_csv('./data/friday_processed.csv'))
data_mean_variance_list.append(pd.read_csv('./data/saturday_processed.csv'))
data_mean_variance_list.append(pd.read_csv('./data/sunday_processed.csv'))

# Import the schedule of the transportation for each day
data_schedule_list = []
data_schedule_list.append(pd.read_csv('./data/monday_schedule.csv'))
data_schedule_list.append(pd.read_csv('./data/tuesday_schedule.csv'))
data_schedule_list.append(pd.read_csv('./data/wednesday_schedule.csv'))
data_schedule_list.append(pd.read_csv('./data/thursday_schedule.csv'))
data_schedule_list.append(pd.read_csv('./data/friday_schedule.csv'))
data_schedule_list.append(pd.read_csv('./data/saturday_schedule.csv'))
data_schedule_list.append(pd.read_csv('./data/sunday_schedule.csv'))

# Import the station with multiple lines
transfer_station_list = pickle.load( open( "./data/transfer.p", "rb" ) )

# Import the adjacent matrix computed
adj_map = pickle.load(open("./data/connection.p","rb"))

Additionally, call the functions built in the previous sections that will be needed to build a routing algorithm. 

In [3]:
# This function is reused from the PART 0.
'''
The function to compute the distance between two sets of (longtitude, latitude).
'''

from math import sin, cos, sqrt, atan2, radians
zurich_longtitude = 8.540192
zurich_latitude = 47.378177
def compute_distance(parameter,longtitude2 = zurich_longtitude,latitude2 = zurich_latitude):
    # approximate radius of earth in km
    R = 6373.0
    longtitude1,latitude1 = parameter
    longtitude1 = float(longtitude1)
    latitude1 = float(latitude1)
    lat1 = radians(latitude1)
    lon1 = radians(longtitude1)
    lat2 = radians(latitude2)
    lon2 = radians(longtitude2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [4]:
# This function is reused from the PART 3.
'''
This function gives the probability that P(X<x) , where X is a gamma distribution random variable
'''
def compute_probability(parameter,x):
    k = parameter[0]
    theta = parameter[1]
    if k*theta*theta < 0.01:
        if k*theta < x:
            return 1.0
        else:
            return 0.0
    dist = scipy.stats.gamma(k, 0, theta)
    return dist.cdf(x)

In [5]:
# This function is reused from the PART 3.
'''
Now we give the function to compute the probability that P(X<Y) where X and Y are both gamma distribution.This cannot be 
compute analytically so we use Monte Carlo simulation to simulate the probability.
'''
def compute_probability_sample(parameter_departure,parameter_arrival,N=1000):
    k_arrival, theta_arrival = parameter_arrival
    k_departure, theta_departure = parameter_departure
    # If it never departures, k_departure will be -1, then we just give 0 for return which means there is no
    # possibility that you catch any train from here
    if k_departure < 0 or theta_departure < 0:
        return 0.0
    if np.isnan(k_departure) or np.isnan(theta_departure):
        return 0.0
    # Build two distribution
    dist_arrival = scipy.stats.gamma(k_arrival, 0, theta_arrival+1e-20)
    dist_departure = scipy.stats.gamma(k_departure, 0, theta_departure+1e-20)
    # Draw samples
    arrival_s = dist_arrival.rvs(size=N)
    departure_s = dist_departure.rvs(size=N)
    # Return the probability by simulation
    return np.sum(arrival_s<departure_s)/N

## Routing algorithm implementation

To increase the efficiency of the routing algorithm, when we search the routes, we only consider the stations which we can use to make a transfer. There is no sense to compute the route possibility if it is not a transfer station or a destination(You won't need to achieve there). For this, we are using `transfer_station_list` and `adjacent_matrix` that were computed and stored in previous sections. 

In [6]:
# Make a list for stations
stations = [int(i) for i in station_data.station_number.values]

In [7]:
# Get a bool list for whether it is a transfer matrix
transfer_boolean = [(i in transfer_station_list) for i in stations]

In [8]:
# Find the station which is isolate (For increase efficiency)
block_station_list = []
for block_index,block_station in enumerate([np.count_nonzero(i-1e9) for i in list(adj_map)]):
    if block_station == 0:
        block_station_list.append(block_index)
print(block_station_list)

[0, 31, 33, 35, 36, 37, 39, 40, 42, 73, 74, 75, 87, 88, 89, 90, 91, 92, 93, 94, 95, 109, 113, 114, 116, 253, 529, 936, 961]


Here, we defined functions that are specific for the routing algorithm. 

In [9]:
def find_route_map(departure_station,arrival_station):
    departure_index = -1
    arrival_index = -1
    max_depth = 4
    # Get the index from the train number
    for i,station in enumerate(stations):
        if station == departure_station:
            departure_index = i
        if station == arrival_station:
            arrival_index = i
        if (departure_index != -1) and (arrival_index != -1):
            break
    # If it is a isolate station, return no route find
    if departure_index in block_station_list or arrival_index in block_station_list:
        return -1
    print ("index",departure_index,arrival_index)
    # Whether already found the solution
    flag = 0
    # Change all 10 to 1 (In this function, we do not care it is walk or public transport)
    adj_map_ = adj_map.copy()
    adj_map_[adj_map_==10.0]=1.0
    # Initialize the result
    result = np.array([[departure_station]])
    final_list_depth = []
    if adj_map_[departure_index,arrival_index]==1:
        #final_list_depth.append(result)
        max_depth = 2
        flag = 1
        #print (final_list_depth)
    # Set the transfer and the arrival as 1 (The other station we do not care)
    transfer_int = np.array(transfer_boolean).astype(np.int).copy()
    transfer_int[arrival_index] = 1.0
    # Depth of this search
    depth = 0

    final_list = []
    while True:
        depth = depth +1
        print (depth)
        result_temp_list = []
        #print (result)
        for i in range(result.shape[0]):
            # Get departure station index
            departure_station = result[i,-1]
            for j,station in enumerate(stations):
                if station == departure_station:
                    departure_index = j
                    break

            # Find the station can be reached
            this_level_station = (adj_map_[departure_index,:] == 1.0).astype(int)*transfer_int
            if this_level_station[arrival_index] == 1:
                # If we found the station, then set the flag = 1 and also update the max_depth 
                # Do not find the transfer which is more than 2 than the shortest transfer
                # People won't take the train has too much transfer 
                if flag==0:
                    max_depth = depth + 2
                flag = 1
                final_list.append(result[i,:].reshape(1,-1))
            # Prepare the data of the next level
            this_level_station[arrival_index] == 0
            station_temp = np.array(stations)[this_level_station.astype(bool)]
            result_temp = np.repeat(result[i,:].reshape(1,-1),len(station_temp.tolist()),axis = 0)
            result_temp = np.concatenate([result_temp,station_temp.reshape(-1,1)], axis = 1)
            result_temp_list.append(result_temp)
        if flag == 1:
            final_list_depth.append(np.concatenate(final_list,axis = 0))
            final_list = []
        #print('result_temp: ',result_temp_list)
        # If already no more transfer, then return
        if result_temp_list == []:
            return final_list_depth
        result = np.concatenate(result_temp_list,axis = 0)
        print('max_depth: ',max_depth)
        # If greater than max_depth, then return
        if depth == min(4,max_depth):
            return final_list_depth

In [10]:
def check_one_line(start_time,route,destination,probability,dayofweek,walk_speed = 1.75):
    # Departure time is precise 
    time_list_mean = [start_time]
    time_list_var = [0.01]
    connect_info = []
    # Pass every step of the route
    for departure_station,arrival_station in zip(route,route[1:]+[destination]):
        departure_index = -1
        arrival_index = -1
        # Find the departure and arrival index
        for i,station in enumerate(stations):
            if station == departure_station:
                departure_index = i
            if station == arrival_station:
                arrival_index = i
            if (departure_index != -1) and (arrival_index != -1):
                break
        if adj_map[departure_index,arrival_index] == 1.0:#walk
            # Compute the walking time
            pos1 = station_data[station_data.station_number == str(departure_station).zfill(7)][["longtitude","latitude"]].as_matrix()[0]
            pos2 = station_data[station_data.station_number == str(arrival_station).zfill(7)][["longtitude","latitude"]].as_matrix()[0]
            d = compute_distance((float(pos1[0]),float(pos1[1])),float(pos2[0]),float(pos2[1]))
            walking_time = d*1000/walk_speed
            # Shift the mean and do not shift the variance
            mean = time_list_mean[-1]+walking_time
            var = time_list_var[-1]
            time_list_mean.append(mean)
            time_list_var.append(var)
            connect_info.append((-1,-1, mean, var))
        if adj_map[departure_index,arrival_index] == 10.0:#transport
            # Filter only keeps the data with relevant day
            data_mean_variance = data_mean_variance_list[dayofweek]
            data_schedule = data_schedule_list[dayofweek]
            # Find the connection between the station
            depart_train = data_schedule[data_schedule.station_id == departure_station]
            arrival_train = data_schedule[data_schedule.station_id == arrival_station]
            connect_train = pd.merge(depart_train, arrival_train, how="inner",on=["train_number","line_id"])
            connect_train = connect_train[connect_train.departuretimeoffsetschedule_x < connect_train.arrivaltimeoffsetschedule_y]
            connect_train_distribution = pd.merge(connect_train, data_mean_variance[data_mean_variance.station_id==departure_station], \
                                                  how="left",on=["train_number","line_id"])
            if connect_train_distribution.shape[0]==0:
                return -1
            # Compute the possibility of the transfer
            theta = time_list_var[-1]/time_list_mean[-1]
            k = time_list_mean[-1]/theta
            connect_train_distribution = connect_train_distribution[connect_train_distribution.departure_k>0]
            connect_train_distribution = connect_train_distribution[connect_train_distribution.departure_theta>0]
            train_frame_probability_temp = connect_train_distribution[['departure_k','departure_theta']].apply(compute_probability_sample, axis=1,args=((k,theta),))
            # Keep the train which satisify the probability low bound
            if train_frame_probability_temp.shape[0]==0:
                return -1
            connect_train_distribution = connect_train_distribution[train_frame_probability_temp>probability]
            connect_train_distribution = connect_train_distribution.sort_values(by = ['line_id',"departuretimeoffsetschedule_y"])
            connect_train_distribution = connect_train_distribution.drop_duplicates(subset = ["line_id"])
            if connect_train_distribution.shape[0] == 0:
                return -1
            connect_train_distribution = connect_train_distribution[["train_number","line_id"]]
            final = pd.merge(connect_train_distribution, data_mean_variance[data_mean_variance.station_id==arrival_station], \
                                                  how="left",on=["train_number","line_id"])
            # Save the information
            final = final.sort_values(by = ["avg(arrivaltimeoffset)"])
            train_number = final.at[0,"train_number"]
            line_id = final.at[0,"line_id"]
            mean = final.at[0,"avg(arrivaltimeoffset)"]
            var = final.at[0,"var(arrivaltimeoffset)"]
            connect_info.append((train_number,line_id, mean, var))
            time_list_mean.append(mean)
            time_list_var.append(var)
            if np.isnan(mean) or np.isnan(var):
                return -1
    return connect_info
            

In [11]:
def find_route(departure_station,arrival_station,departure_time,probability):
    # Change the data to pandas datatime
    time = pd.to_datetime(departure_time,dayfirst=True)
    dayofweek = time.dayofweek
    start_second = time.timestamp()%(24*3600)
    route = find_route_map(departure_station,arrival_station)
    info_list = []
    info_list_station =[]
    for depth_route in route:
        for i in range(depth_route.shape[0]):
            r = depth_route[i]
            info_list_station.append(r)
            print (r)
            info = check_one_line(start_second,list(r),arrival_station,probability,dayofweek,walk_speed = 1.75)
            if(len(info_list) != 0) and (info != -1):
                print(info)
                prev_info = info[0][1][-3:] # first transport_line
                for next_info in info[1:]:
                    if(next_info[0]!= -1): # if it is not a walking 
                        if(prev_info == next_info[1][-3:]):
                            print(prev_info, next_info[1][-3:])
                            info = -1
                    
            if(info == -1):
                info_list_station.pop(-1)
            else:
                info_list.append(info)
                
    print('Finished finding Routes\n')
    return info_list, info_list_station
    

### Example of routing to some destinations

Here is the example of the possible travel we can do from Zürich HB(8503000) to another destination. 

1. 
   - Destination : 8503001
   - Datetime : 21.05.2018 16:15:00 
   - Probability : 90%
   
2. 
    - Destination : 8503001
    - Datetime : 06.06.2018 11:00:00 
    - Probability : 80% 
    
3. 
   - Destination : 8594261
   - Datetime : 21.05.2018 16:15:00 
   - Probability : 90%

In [12]:
starting = 8503000
ending = 8503001

#### Let's first try with the first one:
We want to get to the station whose number is 8503001

In [13]:
station_data[station_data.station_number==str(ending)]

,station_number,longtitude,latitude,name,distance
2380,8503001,8.488940,47.391481,Zürich Altstetten,4.133759


In [14]:
info_list, info_list_station = find_route(starting, ending,'21.05.2018 16:15:00',0.9)

index 10 11
1
max_depth:  2
2
max_depth:  2
[8503000]
[8503000 8502220]
[('85:11:18562:001', 'Zug:18562:S5', 60562.0, 2295.0), ('85:11:18565:001', 'Zug:18565:S5', 62142.0, 3670.0)]
:S5 :S5
[8503000 8502221]
[('85:11:18562:001', 'Zug:18562:S5', 60896.0, 2952.0), ('85:11:18565:001', 'Zug:18565:S5', 62142.0, 3670.0)]
:S5 :S5
[8503000 8502222]
[('85:11:18562:001', 'Zug:18562:S5', 61222.0, 2729.0), ('85:11:18567:001', 'Zug:18567:S5', 63967.0, 14186.0)]
:S5 :S5
[8503000 8502229]
[('85:11:18562:001', 'Zug:18562:S5', 60693.0, 2583.0), ('85:11:18565:001', 'Zug:18565:S5', 62142.0, 3670.0)]
:S5 :S5
[8503000 8503003]
[('85:11:18363:002', 'Zug:18363:S3', 59741.0, 144.0), ('85:11:18362:001', 'Zug:18362:S3', 61543.0, 2836.0)]
:S3 :S3
[8503000 8503006]
[('85:11:18060:001', 'Zug:18060:S', 59688.0, 20295.0), ('85:11:19462:001', 'Zug:19462:S14', 60874.0, 1932.0)]
[8503000 8503016]
[('85:11:1527:002', 'Zug:1527:IC5', 60607.0, 2476.0), ('85:11:2080:001', 'Zug:2080:IR', 62193.0, 4547.0)]
[8503000 8503020]
[

Let's order the list of possible routes by displaying first the ones that take less time.

In [15]:
def getTime(elem):
    return elem[0][-1][2]

In [16]:
full_info = list(zip(info_list,info_list_station))
full_info.sort(key = getTime)
full_info

[([('85:11:18360:002', 'Zug:18360:S3', 59703.0, 945.0)], array([8503000])),
 ([('85:11:18060:001', 'Zug:18060:S', 59367.0, 21591.0),
   ('85:11:18360:002', 'Zug:18360:S3', 59703.0, 945.0)],
  array([8503000, 8503020])),
 ([('85:11:18060:001', 'Zug:18060:S', 59688.0, 20295.0),
   ('85:11:19462:001', 'Zug:19462:S14', 60874.0, 1932.0)],
  array([8503000, 8503006])),
 ([('85:11:18860:001', 'Zug:18860:S8', 59634.0, 1438.0),
   ('85:11:19462:001', 'Zug:19462:S14', 60874.0, 1932.0)],
  array([8503000, 8503129])),
 ([('85:11:1527:002', 'Zug:1527:IC5', 60607.0, 2476.0),
   ('85:11:2080:001', 'Zug:2080:IR', 62193.0, 4547.0)],
  array([8503000, 8503016])),
 ([('85:11:18961:001', 'Zug:18961:S9', 60159.0, 1887.0),
   ('85:11:19464:001', 'Zug:19464:S14', 62684.0, 3397.0)],
  array([8503000, 8503127])),
 ([('85:11:18961:001', 'Zug:18961:S9', 59952.0, 1436.0),
   ('85:11:19464:001', 'Zug:19464:S14', 62684.0, 3397.0)],
  array([8503000, 8503128]))]

We keep only the 5 fastest. 

In [17]:
if len(full_info)>5:
    full_info = full_info[0:5]

Let's extract the information about the means of transportation used (if by walk or using the public transportation)

In [18]:
howImove = []
lines = []
for line in full_info:
    line_info = line[0]
    means = []
    for i in range(len(line_info)):
        if line_info[i][0] == -1 and line_info[i][1] == -1:
            means.append('walk')
        else:
            means.append('transit')
    howImove.append(means)
    myLine = list(line[-1])
    myLine.append(ending)
    lines.append(myLine)
howImove
    

[['transit'],
 ['transit', 'transit'],
 ['transit', 'transit'],
 ['transit', 'transit'],
 ['transit', 'transit']]

And now retrieve the latitude and longitude for each station in every route computeted.

In [19]:
Stations = []
for line in lines:
    lineStations = []
    for stat in line:
        
        (lat, long) = float(station_data[station_data['station_number']==str(stat)]['latitude'].item()),\
                    float(station_data[station_data['station_number']==str(stat)]['longtitude'].item())
        lineStations.append((lat,long))
    
    Stations.append(lineStations)
        

In [20]:
Stations

[[(47.378177, 8.540192), (47.391481, 8.48894)],
 [(47.378177, 8.540192), (47.385195, 8.517106), (47.391481, 8.48894)],
 [(47.378177, 8.540192), (47.411529, 8.544115), (47.391481, 8.48894)],
 [(47.378177, 8.540192), (47.412717, 8.591911), (47.391481, 8.48894)],
 [(47.378177, 8.540192), (47.450383, 8.562386), (47.391481, 8.48894)]]

## Visualization 

Revert the list since we want to display the fastest on top:

In [21]:
howImove.reverse()
howImove

[['transit', 'transit'],
 ['transit', 'transit'],
 ['transit', 'transit'],
 ['transit', 'transit'],
 ['transit']]

We'll use gmap

In [22]:
import gmaps
key = 'AI...'
 if key == 'AI...':
    raise Exception('You need to put your Gmap key here!')
gmaps.configure(api_key=key)

Now we can visualize the first five routes proposed by the algorithm:

In [23]:
fig = gmaps.figure(center=(47.378177, 8.540192), zoom_level=12)

colors = ['blue', 'green', 'black', 'purple', 'red']

a = 0

  
for stat in reversed(Stations):
    for i in range(len(stat)):
        travel = stat[i:2+i] if len(stat[i:2+i]) > 1 else []
        
        if travel != []:
            
            if(howImove[a][i] == 'transit'):
                #print(colors[a])
                station2station = gmaps.directions_layer(travel[0], travel[1],
                                             stroke_color=colors[a], travel_mode='TRANSIT', show_markers=False)
            else:
                if (howImove[a][i]=='walk'):
                    print('walk')
                    station2station = gmaps.directions_layer(travel[0], travel[1],
                                             stroke_color=colors[a], travel_mode='WALKING', show_markers=False)
                    
    
            fig.add_layer(station2station)
    a = a+1

symbols = gmaps.symbol_layer(
        [Stations[0][0], Stations[0][1]], fill_color=['blue', 'red'], scale = 7, stroke_color=['blue', 'red'] )

fig.add_layer(symbols)

fig

Figure(layout=FigureLayout(height='420px'))

This is the captured image for above cell. 

<img src="./images/route1.jpeg">
</img>

Here, we can see the train number, line id, the mean time of arrival and variance for each of the stations reached during every route.

In [24]:
[i[0] for i in full_info]

[[('85:11:18360:002', 'Zug:18360:S3', 59703.0, 945.0)],
 [('85:11:18060:001', 'Zug:18060:S', 59367.0, 21591.0),
  ('85:11:18360:002', 'Zug:18360:S3', 59703.0, 945.0)],
 [('85:11:18060:001', 'Zug:18060:S', 59688.0, 20295.0),
  ('85:11:19462:001', 'Zug:19462:S14', 60874.0, 1932.0)],
 [('85:11:18860:001', 'Zug:18860:S8', 59634.0, 1438.0),
  ('85:11:19462:001', 'Zug:19462:S14', 60874.0, 1932.0)],
 [('85:11:1527:002', 'Zug:1527:IC5', 60607.0, 2476.0),
  ('85:11:2080:001', 'Zug:2080:IR', 62193.0, 4547.0)]]

Let's try again with the second example:

In [25]:
###########################Find Routes#############################
starting = 8503000
ending = 8503001
info_list, info_list_station = find_route(starting, ending, '06.06.2018 11:00:00',0.8)
###########################Order them##############################
full_info = list(zip(info_list,info_list_station))
full_info.sort(key = getTime)
if len(full_info)>5:
    full_info = full_info[0:5]
#######################Retrieve Information########################
howImove = []
lines = []
for line in full_info:
    line_info = line[0]
    means = []
    for i in range(len(line_info)):
        if line_info[i][0] == -1 and line_info[i][1] == -1:
            means.append('walk')
        else:
            means.append('transit')
    howImove.append(means)
    myLine = list(line[-1])
    myLine.append(ending)
    lines.append(myLine)

Stations = []
for line in lines:
    lineStations = []
    for station in line:
        
        (lat, long) = float(station_data[station_data['station_number']==str(station)]['latitude'].item()),\
                    float(station_data[station_data['station_number']==str(station)]['longtitude'].item())
        lineStations.append((lat,long))
    
    Stations.append(lineStations)
howImove.reverse()


index 10 11
1
max_depth:  2
2
max_depth:  2
[8503000]
[8503000 8502220]
[('85:11:18540:001', 'Zug:18540:S5', 40748.0, 1143.0), ('85:11:18543:001', 'Zug:18543:S5', 42273.0, 1124.0)]
:S5 :S5
[8503000 8502221]
[('85:11:18540:001', 'Zug:18540:S5', 41076.0, 1131.0), ('85:11:18543:001', 'Zug:18543:S5', 42273.0, 1124.0)]
:S5 :S5
[8503000 8502222]
[('85:11:18540:001', 'Zug:18540:S5', 41393.0, 482.0), ('85:11:18545:001', 'Zug:18545:S5', 44104.0, 5824.0)]
:S5 :S5
[8503000 8502229]
[('85:11:18540:001', 'Zug:18540:S5', 40873.0, 936.0), ('85:11:18543:002', 'Zug:18543:S5', 42303.0, 2557.0)]
:S5 :S5
[8503000 8503003]
[('85:11:18341:001', 'Zug:18341:S3', 39949.0, 3343.0), ('85:11:18340:001', 'Zug:18340:S3', 41739.0, 7519.0)]
:S3 :S3
[8503000 8503006]
[('85:11:18060:001', 'Zug:18060:S', 59646.0, 2116.0), ('85:11:19462:001', 'Zug:19462:S14', 60885.0, 1641.0)]
[8503000 8503016]
[8503000 8503020]
[('85:11:18060:001', 'Zug:18060:S', 59331.0, 1693.0), ('85:11:18360:001', 'Zug:18360:S3', 59736.0, 1665.0)]
[8

In [26]:
print('Routing information')
[i[0] for i in full_info]


Routing information


[[('85:11:18340:001', 'Zug:18340:S3', 41739.0, 7519.0)],
 [('85:11:18840:001', 'Zug:18840:S8', 41632.0, 5193.0),
  ('85:11:19442:001', 'Zug:19442:S14', 42856.0, 359.0)],
 [('85:11:18941:001', 'Zug:18941:S9', 42142.0, 2000.0),
  ('85:11:19444:001', 'Zug:19444:S14', 44697.0, 2071.0)],
 [('85:11:18941:001', 'Zug:18941:S9', 41944.0, 1969.0),
  ('85:11:19444:001', 'Zug:19444:S14', 44697.0, 2071.0)],
 [('85:11:18060:001', 'Zug:18060:S', 59331.0, 1693.0),
  ('85:11:18360:001', 'Zug:18360:S3', 59736.0, 1665.0)]]

In [27]:
fig = gmaps.figure(center=(47.378177, 8.540192), zoom_level=12)

colors = ['blue', 'green', 'black', 'purple', 'red']

a = 0

  
for stat in reversed(Stations):
    for i in range(len(stat)):
        travel = stat[i:2+i] if len(stat[i:2+i]) > 1 else []
        
        if travel != []:
            
            if(howImove[a][i] == 'transit'):
                #rint(colors[a])
                station2station = gmaps.directions_layer(travel[0], travel[1],
                                             stroke_color=colors[a], travel_mode='TRANSIT', show_markers=False)
            else:
                if (howImove[a][i]=='walk'):
                    print('walk')
                    station2station = gmaps.directions_layer(travel[0], travel[1],
                                             stroke_color=colors[a], travel_mode='WALKING', show_markers=False)
                    
    
            fig.add_layer(station2station)
    a = a+1

symbols = gmaps.symbol_layer(
        [Stations[0][0], Stations[0][-1]], fill_color=['blue', 'red'], scale = 7, stroke_color=['blue', 'red'] )

fig.add_layer(symbols)

fig

Figure(layout=FigureLayout(height='420px'))

This is the captured image for above cell. 

<img src="./images/route2.jpeg">
</img>

This is the last example.. 

Since these two stations are connected by so many routes, this time we are only visualizing the 3 bests.

In [28]:
###########################Find Routes#############################
starting = 8503000
ending = 8594261
info_list, info_list_station = find_route(starting, ending, '21.05.2018 16:15:00',0.9)
###########################Order them##############################
full_info = list(zip(info_list,info_list_station))
full_info.sort(key = getTime)
if len(full_info)>3:
    full_info = full_info[0:3]
#######################Retrieve Information########################
howImove = []
lines = []
for line in full_info:
    line_info = line[0]
    means = []
    for i in range(len(line_info)):
        if line_info[i][0] == -1 and line_info[i][1] == -1:
            means.append('walk')
        else:
            means.append('transit')
    howImove.append(means)
    myLine = list(line[-1])
    myLine.append(ending)
    lines.append(myLine)

Stations = []
for line in lines:
    lineStations = []
    for station in line:
        
        (lat, long) = float(station_data[station_data['station_number']==str(station)]['latitude'].item()),\
                    float(station_data[station_data['station_number']==str(station)]['longtitude'].item())
        lineStations.append((lat,long))
    
    Stations.append(lineStations)
howImove.reverse()



index 10 1014
1
max_depth:  4
2
max_depth:  4
3
max_depth:  5
4
max_depth:  5
[8503000 8503128 8587653]
[8503000 8503147 8591065]
[('85:11:18363:002', 'Zug:18363:S3', 60048.0, 415.0), (-1, -1, 60052.00519845217, 415.0), ('85:773:24292-01752-1', 'Bus:85:773:760:760', 61607.0, 1718.0)]
[8503000 8502220 8503128 8587653]
[('85:11:18562:001', 'Zug:18562:S5', 60562.0, 2295.0), ('85:11:19465:001', 'Zug:19465:S14', 62766.0, 9435.0), (-1, -1, 62790.05738697158, 9435.0), ('85:773:24293-01752-1', 'Bus:85:773:760:760', 63501.0, 3427.0)]
[8503000 8502221 8503128 8587653]
[('85:11:18562:001', 'Zug:18562:S5', 60896.0, 2952.0), ('85:11:19467:001', 'Zug:19467:S14', 64532.0, 8197.0), (-1, -1, 64556.05738697158, 8197.0), ('85:773:24551-01752-1', 'Bus:85:773:760:760', 65322.0, 6244.0)]
[8503000 8502222 8503128 8587653]
[('85:11:18562:001', 'Zug:18562:S5', 61222.0, 2729.0), ('85:11:19467:001', 'Zug:19467:S14', 64532.0, 8197.0), (-1, -1, 64556.05738697158, 8197.0), ('85:773:24551-01752-1', 'Bus:85:773:760:7

[('85:11:18363:002', 'Zug:18363:S3', 60272.0, 365.0), (-1, -1, 60284.682898431325, 365.0), ('85:773:24559-01752-1', 'Bus:85:773:748:748', 61009.0, 8604.0), ('85:773:24255-01752-1', 'Bus:85:773:760:760', 61733.0, 5868.0)]
[8503000 8503306 8588314 8590591]
[('85:11:18363:002', 'Zug:18363:S3', 60272.0, 365.0), (-1, -1, 60284.682898431325, 365.0), ('85:773:24559-01752-1', 'Bus:85:773:748:748', 61058.0, 8178.0), ('85:773:24255-01752-1', 'Bus:85:773:760:760', 61733.0, 5868.0)]
[8503000 8503310 8503128 8587653]
[('85:11:18060:001', 'Zug:18060:S', 59904.0, 20801.0), ('85:11:18963:001', 'Zug:18963:S9', 61785.0, 5756.0), (-1, -1, 61809.05738697158, 5756.0), ('85:773:24293-01752-1', 'Bus:85:773:760:760', 63501.0, 3427.0)]
[8503000 8503310 8503147 8591065]
[('85:11:18060:001', 'Zug:18060:S', 59904.0, 20801.0), ('85:11:18963:001', 'Zug:18963:S9', 61565.0, 3414.0), (-1, -1, 61569.00519845217, 3414.0), ('85:773:24256-01752-1', 'Bus:85:773:760:760', 62505.0, 2014.0)]
[8503000 8503310 8590620 8591065]


In [29]:
print('Routing information')
[i[0] for i in full_info]

Routing information


[[('85:11:18961:001', 'Zug:18961:S9', 59952.0, 1436.0),
  (-1, -1, 59976.05738697158, 1436.0),
  ('85:773:24291-01752-1', 'Bus:85:773:760:760', 60817.0, 2587.0)],
 [('85:11:18961:001', 'Zug:18961:S9', 59952.0, 1436.0),
  (-1, -1, 59976.05738697158, 1436.0),
  ('85:773:24501-01752-1', 'Bus:85:773:748:748', 60480.0, 1406.0),
  ('85:773:24291-01752-1', 'Bus:85:773:760:760', 60817.0, 2587.0)],
 [('85:11:18961:001', 'Zug:18961:S9', 59952.0, 1436.0),
  (-1, -1, 59976.05738697158, 1436.0),
  ('85:773:24501-01752-1', 'Bus:85:773:748:748', 60439.0, 1043.0),
  ('85:773:24291-01752-1', 'Bus:85:773:760:760', 60817.0, 2587.0)]]

In [30]:
fig = gmaps.figure(center=(47.378177, 8.540192), zoom_level=12)

colors = ['blue', 'green', 'red']

a = 0

  
for stat in reversed(Stations):
    for i in range(len(stat)):
        travel = stat[i:2+i] if len(stat[i:2+i]) > 1 else []
        
        if travel != []:
            if(howImove[a][i] == 'transit'):
                station2station = gmaps.directions_layer(travel[0], travel[1],
                                             stroke_color=colors[a], travel_mode='TRANSIT', show_markers=False)
            else:
                if (howImove[a][i]=='walk'):
                    print('walk')
                    station2station = gmaps.directions_layer(travel[0], travel[1],
                                             stroke_color=colors[a], travel_mode='WALKING', show_markers=False)
                    
    
            fig.add_layer(station2station)
    a = a+1

symbols = gmaps.symbol_layer(
        [Stations[0][0], Stations[0][-1]], fill_color=['blue', 'red'], scale = 7, stroke_color=['blue', 'red'] )

fig.add_layer(symbols)

fig

walk
walk
walk


Figure(layout=FigureLayout(height='420px'))

This is the captured image for above cell. 

<img src="./images/route3.jpeg">
</img>

## Validation

In order to validate the results, we can check the best route we had found in term of transit time (the red one) and compare it with the one outputted by Google Gmap. Hereafter is presented the first one (on Monday, departing at 16,15 from Zurich HB to Zurich Altstetten):

<img src="./images/route_validation1.jpeg">
</img>

Please notice that our best route overlaps with the one Google suggests. However, some of our routes are making a turistic tour of Zurich. Indeed, those routes, even if they travel a lot, are arriving earlier than the rest of the all other possible routes. It might be preferable to wait for the same direct train instead of taking other routes, however, in our algorithm we did not implement the case for waiting at the station for the next train. Hence, the algorithm will prefer to take a travel instead of waiting. 

Hereafter, GMap results for a route starting on Wednesday, departing at 11:00 from Zurich HB to Zurich Altstetten:

<img src="./images/route_validation2.jpeg">
</img>

Even though the destination is the same, some of the routes are different from the previous example since the calculated mean and variance for trip can be different from day to day. Still, our best route (red one) is the same as the one suggested by Google.

Hereafter, GMap results for a route starting on Monday, departing at 16:15 from Zurich HB to Zurich Dübendorf, Hochbord Nord:

<img src="./images/route_validation3.jpeg">
</img>

In this case, the fastest route we've computed doesn't match with Google's suggestion. Indeed, Google suggests to take a bus which, from schedule, will take a shorter time. Our algorithm, instead, prefers to take a train. Generally, buses are more prone to be delayed by traffic, so it is more likely to arrive later than the schedule time. Since we are also considering the probability of arriving, it makes sense to suggest to take a train.

## Discussion

**Pros:**


The algorithm we designed both for routing and isochrone map is agnostic to the departing station hence we're not forced to start from Zurich HB.
Cleaned dataset divided into 7 days to ensure a easier computation and modularity.
Modularity is ensured: changing in the dataset only require to recompute the distributions, adding special days only require to load them and compute mean and variance.


**Cons:**


The algorithm to compute the isochrone map could be slow due to the exponential nature of the recursive algorithm upon it was built.
Heavily relying on GoogleMaps for visualization: nowadays is a fast-changing enviroment which is turning to a paid service.
Not taking into account "bank holidays".


**Possible improvements:**


For the moment we only take into account, for walking, stations in a radius of 100 meters from the arrival point, it's possible to improve this by putting no-limits in the distance we can walk and optimize over it.
The current used PDF (the gamma function) is a first guess, it can be improved with different models, for example, a Poisson distribution.
It's possible to implement a live-version using streaming data in order to compute the distribution over a moving time window.